In [1]:
import pymysql

In [4]:
# connection
mydb = pymysql.connect(host="localhost", user="root", password="", db="sqdb")
query = mydb.cursor()

In [238]:
query.execute("SELECT * FROM payment_collection_organization_master WHERE Payment_id=2013022045001")
result = query.fetchall() # fetchone(), fetchmany(1,2)

# for row in result:
#     print(row)

In [239]:
query.execute("SELECT a.Payment_date, SUM(b.Pay_amount) as Pay_amount FROM payment_collection_organization_master a INNER JOIN payment_collection_organization_detail b ON(a.Payment_id = b.Payment_id) WHERE a.Transaction_posting_type=2 AND a.Payment_date<'2018-01-01' GROUP BY a.Payment_date")
result = query.fetchall()

print(len(result))

# for row in result:
#     print(row[0], row[1])

1233


In [215]:
import numpy as np
import plotly as py
import plotly.express as px
import plotly.graph_objs as go
import pandas as pd

In [240]:
df = pd.DataFrame( [[ij for ij in i] for i in result] )
df

,0,1
0,2012-12-27,33000.00
1,2013-02-04,4880.00
2,2013-02-19,38277.00
3,2013-04-07,70000.00
4,2013-04-18,15000.00
...,...,...
1228,2017-12-26,1199481.15
1229,2017-12-27,685697.00
1230,2017-12-28,4452450.87
1231,2017-12-30,386455.00


In [241]:
df.rename(columns={0: 'Date', 1: 'Payment Amount'}, inplace=True);
df.head()

,Date,Payment Amount
0,2012-12-27,33000.00
1,2013-02-04,4880.00
2,2013-02-19,38277.00
3,2013-04-07,70000.00
4,2013-04-18,15000.00


In [243]:
fig = px.bar(df.tail(90), x="Date", y="Payment Amount", barmode='group', height=600)
fig.update_layout(title_text='Payment Report Last 90 days', plot_bgcolor='rgb(230, 230, 230)')
fig.show()

In [244]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Date'].tail(90), y = df['Payment Amount'].tail(30), mode='lines+markers',name='Total Cases'))
fig.update_layout(title_text='Payment Report last 90 days', plot_bgcolor='rgb(230, 230, 230)')
fig.show()

In [245]:
# Prediction
from fbprophet import Prophet

In [253]:
payment = df.groupby('Date').sum()['Payment Amount'].reset_index()

In [254]:
payment.columns = ['ds','y']
payment['ds'] = pd.to_datetime(payment['ds'])
payment.tail()

,ds,y
1228,2017-12-26,1199481.15
1229,2017-12-27,685697.00
1230,2017-12-28,4452450.87
1231,2017-12-30,386455.00
1232,2017-12-31,242394.00


In [255]:
m = Prophet(interval_width=0.95)
m.fit(payment)
future = m.make_future_dataframe(periods=7)
future.tail()

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds
1235,2018-01-03
1236,2018-01-04
1237,2018-01-05
1238,2018-01-06
1239,2018-01-07


In [262]:
#predicting the future with date, and upper and lower limit of y value
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
1235,2018-01-03,1.950104e+06,-1.697880e+06,5.528223e+06
1236,2018-01-04,2.249539e+06,-1.739895e+06,6.134809e+06
1237,2018-01-05,9.513011e+05,-2.501316e+06,4.618663e+06
1238,2018-01-06,8.029053e+05,-2.851183e+06,4.707175e+06
1239,2018-01-07,2.104343e+06,-1.689881e+06,5.867348e+06


In [257]:
from fbprophet.plot import plot_plotly, plot_components_plotly
plot_plotly(m, forecast)

In [263]:
plot_components_plotly(m, forecast)